In [19]:
import pandas
import warnings
import yfinance

from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns
from pypfopt.discrete_allocation import DiscreteAllocation, get_latest_prices

warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=DeprecationWarning)

def calculate_metrics(ticker, start_date, end_date):
    # Fetch historical data
    stock_data = yfinance.download(ticker, start=start_date, end=end_date)

    # Calculate daily returns
    stock_data['Daily_Return'] = stock_data['Adj Close'].pct_change()

    # Calculate volatility (annualized standard deviation of daily returns)
    volatility = stock_data['Daily_Return'].std() * (252 ** 0.5)

    # Calculate cumulative returns
    cumulative_returns = (1 + stock_data['Daily_Return']).cumprod() - 1

    # Calculate drawdowns
    cumulative_max = cumulative_returns.cummax()
    drawdowns = (cumulative_returns - cumulative_max) / (cumulative_max + 1)

    # Calculate metrics
    total_return = cumulative_returns.iloc[-1]
    max_drawdown = drawdowns.min()
    avg_annual_return = cumulative_returns.mean() * 252
    sharpe_ratio = avg_annual_return / volatility

    return {
        'Ticker': ticker,
        'Total Return': total_return,
        'Volatility': volatility,
        'Max Drawdown': max_drawdown,
        'Average Annual Return': avg_annual_return,
        'Sharpe Ratio': sharpe_ratio
    }

stocks = ['AAPL', 'GS', 'IBM', 'INTC', 'JNJ', 'JPM', 'MS', 'TRV', 'GOOG', 'TSLA']
start_date = '2019-01-01'
end_date = '2022-01-01'

metrics_data = []
for ticker in stocks:
    metrics = calculate_metrics(ticker, start_date, end_date)
    metrics_data.append(metrics)

# Create a DataFrame to display results
metrics_df = pandas.DataFrame(metrics_data)
metrics_df

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


,Ticker,Total Return,Volatility,Max Drawdown,Average Annual Return,Sharpe Ratio
0,AAPL,3.633111,0.341193,-0.314273,375.470536,1100.464372
1,GS,1.365048,0.361208,-0.456210,140.445203,388.820433
2,IBM,0.405745,0.295225,-0.389801,59.244050,200.674425
3,INTC,0.178846,0.390316,-0.345265,48.563479,124.421085
4,JNJ,0.449921,0.215700,-0.273662,54.146698,251.027628
5,JPM,0.746789,0.354104,-0.436265,84.082907,237.452707
6,MS,1.636927,0.388302,-0.513258,150.613029,387.875854
7,TRV,0.432674,0.320436,-0.462790,52.918213,165.144352
8,GOOG,1.766735,0.295129,-0.307901,164.216168,556.421522
9,TSLA,16.038243,0.669946,-0.606265,1272.419346,1899.286510


In [21]:
df = yfinance.download(stocks, start=start_date)["Close"]

# Calculate Expected annualized returns
mu = expected_returns.mean_historical_return(df)
# Calculate annualized sample covariance matrix returns
S = risk_models.sample_cov(df)

# optimize for the MAX sharpe ratio
# create the efficient frontier object
efficient_frontier = EfficientFrontier(mu, S)
weights = efficient_frontier.max_sharpe()

#round small marginal weights to 0
cleaned_weights = efficient_frontier.clean_weights()

cleaned_weights_df = pandas.DataFrame([cleaned_weights])
cleaned_weights_df


[*********************100%%**********************]  10 of 10 completed


,AAPL,GOOG,GS,IBM,INTC,JNJ,JPM,MS,TRV,TSLA
0,0.64392,0.16508,0.01092,0.0,0.0,0.0,0.0,0.0,0.01184,0.16824
